In [1]:
!pip install azure-storage-blob

FileNotFoundError: [Errno 2] No such file or directory: 'null/Users/ncassonet.ext'

In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

# accessible via Paramètres - Points de terminaison du compte de stockage
account_name = "allocinestorage"  
account_url = f"https://{account_name}.blob.core.windows.net"
container_name = "scraping-container"
blob_name = "allocine.csv"  # ou "dossier/allocine.csv" si dans un sous-dossier
#blob_name = "allocine_20250418_115000.csv"

# URL du fichier ciblé : https://allocinestorage.blob.core.windows.net/scraping-container/allocine.csv
blob_url = f"{account_url}{container_name}/{blob_name}"
#print(blob_url)

# trouvé dans allocinestorage => securite + reseau => cle d'acces
account_key = "..."

blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
blob_downloader = blob_client.download_blob()
downloaded_blob = blob_downloader.readall()

import datetime as dt
from zoneinfo import ZoneInfo

dataframe = None

try:
    
    print(dt.datetime.now(ZoneInfo("Europe/Paris")).strftime("%A %d %B %Y, %H:%M:%S"))

    dataframe = pd.read_csv(io.BytesIO(downloaded_blob), encoding="utf-8-sig", encoding_errors="replace") 
    first_line = dataframe.columns
    first_value = first_line.values[0]
    
        # c'est MAAAAAL de faire comme ça !!!
    first_value = first_value[2:]
    dataframe.columns.values[0] = first_value

except Exception as ex :
    print( ex.__doc__ )

    
if dataframe :
    print(dataframe.head())

Variante si le fichier est déjà dans ML Studio

In [2]:
from azureml.core import Workspace

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
import pandas as pd
import io
import requests

ws = Workspace.from_config()

# Connexion + chargement du dataset v2
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# Lister toutes les versions du dataset "allocine_films"
assets = ml_client.data.list(name="data_allocine_cleaned")
latest_asset = sorted(assets, key=lambda x: x.version, reverse=True)[0]

print(f"latest_asset : version = {latest_asset.version}")

data_asset  = ml_client.data.get(name="data_allocine_cleaned", version=str(latest_asset.version))

df = pd.read_csv(data_asset.path)
df.head()